### Market data refresh 

### Input Description

RAW OHLC data.

### Output  

Clean OHLC data in a hdf store

### Operations

This code takes a financial market data file and runs it through a processing pipeline. The following operations are carried out :

- Localise the time data to market time
- Merge with existing RAW data based on datetime
- Save the resulting RAW data to HDF5

In [1]:
#!pip install --upgrade "../../quantutils"
!pip install "../../marketinsights-price-aggregator"
import json, os, pandas
import quantutils.dataset.pipeline as ppl
from quantutils.api.datasource import MarketDataStore
import MIPriceAggregator.connectors as connectors
import pandas as pd
import numpy as np
import time
from datetime import datetime, date, timedelta

import warnings
from tables import NaturalNameWarning
warnings.filterwarnings('ignore', category=NaturalNameWarning)
from tqdm import tqdm

Processing /home/cwilkin/Development/repos/marketinsights-price-aggregator
  Preparing metadata (setup.py) ... done
  Created wheel for MIPriceAggregator: filename=MIPriceAggregator-1.0.0-py3-none-any.whl size=7730 sha256=7f81cbee6b65d5623b1a327a0f87fde2860152098abd4148706f01ff73ed7368
  Stored in directory: /home/cwilkin/.cache/pip/wheels/a3/66/dc/3a75635dd1cbb1bf931d0df38e458b1890be1666bbb5b88c7f
Successfully built MIPriceAggregator
  Attempting uninstall: MIPriceAggregator
    Found existing installation: MIPriceAggregator 1.0.0
    Uninstalling MIPriceAggregator-1.0.0:
      Successfully uninstalled MIPriceAggregator-1.0.0


In [2]:
def getConnector(connClass, connName, tz, options):
    connectorClass = getattr(connectors, connClass)
    connectorInstance = connectorClass(connName, tz, options)
    return connectorInstance

def fetchHistoricalData(mds, ds_file, start="1979-01-01", end="2050-01-01", records=200, delta=False, newOnly=False):

    # Loop over datasources...
    # TODO: In chronological order

    datasources = json.load(open(ds_file))
    
    data = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'ID']))
    if mds is not None:
        mdsKeys = mds.getKeys()
    else:
        mdsKeys = []
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:

            for source in tqdm(market["sources"]):
                
                if not newOnly or not source["name"] in mdsKeys:
                    
                    if delta:
                        try:
                            start = mds.aggregate([source["name"]], source["sample_unit"]).index[-1].strftime('%Y-%m-%d')
                        except:
                            print("Could not find " + source["name"])
                            start = "1979-01-01"

                        newData = dataConnector.getData(market, source, start)
                    else:
                        newData = dataConnector.getData(market, source, start, end, records)

                    if newData is not None:

                        print("Adding " + source["name"] + " to " + market["name"] + " table")

                        if mds is not None:  
                            mds.append(source["name"], newData, source["sample_unit"], update=True)

                        data = ppl.merge(data, newData)

    return data


In [3]:
def fetchOptionData(datasources, start="1979-01-01", end="2050-01-01"):
            
    options = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'epic']))
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:

            for optionChain in tqdm(market["optionChains"]):
                
                if "options" not in optionChain:
                
                    optionData = dataConnector.getOptions(optionChain, appendUnderlying=False, start=start, end=end)

                    if optionData is not None:                    
                        print("Adding " + optionChain["name"] + " to " + market["name"] + " table")    

                        optionChain["options"] = json.loads(optionData.reset_index()[["epic","instrumentName","strike","type"]].to_json(orient="records"))
                        options = ppl.merge(options, optionData)                                    
        
                        market["sources"].extend(json.loads(optionData.reset_index().assign(name=lambda x: x["epic"]).assign(sample_unit="D")[["name", "sample_unit"]].to_json(orient="records")))
    
    options = options.sort_values(ascending=[False, True, True], by=["Date_Time", "strike", "type"]).dropna()
    return options, datasources
    
def getOptionChains(datasource, root):
    
    dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])
    
    chains = dataConnector.getOptionChains(root)
    
    # Update expiry dates
    dates = []
    for _,chain in chains.iterrows():
        if np.isnan(chain["expiry"]):
            info = dataConnector.getOptionInfo(chain["name"])
            dates.append(info["contractExpirationDate"])
        else:
            dates.append(str(date.today() + timedelta(int(chain["expiry"]) + 1)))
    chains["expiry"] = dates
    
    return chains

def appendOptionChainPrices(mds, ds_file):
    
    datasources = json.load(open(ds_file))
    
    options = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'epic']))
    data = pd.DataFrame()
    mdsKeys = mds.getKeys()
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:

            for optionChain in tqdm(market["optionChains"]):
                
                # Get todays prices from optionChain
                optionData = dataConnector.getOptions(optionChain, appendUnderlying=False)
                                        
                if optionData is not None:
                    
                    print("Adding " + optionChain["name"] + " to " + market["name"] + " table")
                                        
                    df = optionData.replace(np.nan, 0).sort_values(["expiry","strike"]).reset_index()                    
                    for i,option in df.iterrows():
                        newData = df.iloc[i:i+1][["Date_Time","Open","High","Low","Close","Volume","OpenInterest"]].set_index("Date_Time")
                        newData = ppl.localize(newData, datasource["timezone"], datasource["timezone"])
                        newData[['Volume', 'OpenInterest']] = newData[['Volume', 'OpenInterest']].astype(int)
                        if mds is not None and option["epic"] in mdsKeys:
                            print("Adding " + option["epic"] + " to " + market["name"] + " table")
                            mds.append(option["epic"], newData, "D", update=True)
                    
                    options = ppl.merge(options, optionData)
    return options

def appendOptionChainPrices(mds, ds_file):
    
    datasources = json.load(open(ds_file))
    mdsKeys = mds.getKeys()
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:
            
            options = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'ID']))

            for optionChain in tqdm(market["optionChains"]):
                
                # Get todays prices from optionChain
                optionData = dataConnector.getOptions(optionChain, appendUnderlying=False)
                                        
                if optionData is not None:
                                                        
                    optionData = optionData.replace(np.nan, 0).sort_values(["expiry","strike"])[["Open","High","Low","Close","Volume","OpenInterest"]]              
                    
                    options = ppl.merge(options, optionData)
            
            if mds is not None and market["name"] in mdsKeys:
                print("Adding " + optionChain["name"] + " to " + market["name"] + " table")
                mds.append(market["name"], options, "D", update=True, debug=True)
                
    return options


In [12]:
ds_file = "../datasources/datasources_BarChartOption.json"
mds = MarketDataStore(location="../datastore")
data = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['Date_Time', 'ID']))

datasources = json.load(open(ds_file))
    
mdsKeys = mds.getKeys()

for datasource in datasources:
    
    for market in datasource["markets"]:

        for source in tqdm(market["sources"]):

                sourceData = mds.get(source["name"]).assign(ID=source["name"]).reset_index().set_index(["Date_Time", "ID"])
                data = ppl.merge(data, sourceData)

data

  4%|▍         | 118/3117 [00:11<09:33,  5.23it/s]

KeyboardInterrupt: 

In [6]:
data

Open   High    Low  Close  Volume  \
Date_Time                 ID                                              
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23   6.23   6.23       0   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13   6.13   6.13       0   
2020-01-01 00:00:00+00:00 CLZ2|540C    NaN    NaN    NaN    NaN       0   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35   6.35   6.35       0   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21   6.21   6.21       0   
...                                    ...    ...    ...    ...     ...   
2022-10-04 00:00:00+00:00 CLZ3|940C   7.58   7.58   7.58   7.58       0   
                          CLZ3|945C   7.46   7.46   7.46   7.46       0   
                          CLZ3|950C   7.33   7.21   7.33   7.33       0   
                          CLZ3|950P  27.49  27.49  28.04  27.49       0   
                          CLZ3|990C   6.43   6.43   6.43   6.43       0   

                                     OpenInterest  
Date_Time                 ID                       
2019-12-30 00:00:00+00:00 CLZ2|540C           100  
2019-12-31 00:00:00+00:00 CLZ2|540C           100  
2020-01-01 00:00:00+00:00 CLZ2|540C             0  
2020-01-02 00:00:00+00:00 CLZ2|540C           100  
2020-01-03 00:00:00+00:00 CLZ2|540C           100  
...                                           ...  
2022-10-04 00:00:00+00:00 CLZ3|940C             1  
                          CLZ3|945C            82  
                          CLZ3|950C          5534  
                          CLZ3|950P             0  
                          CLZ3|990C             1  

[509539 rows x 6 columns]

In [9]:
data["2021-09-09":"2022-06-01"]

Open   High    Low  Close  Volume  \
Date_Time                 ID                                               
2021-09-09 00:00:00+00:00 CLF3|1000C   0.72   0.72   0.72   0.72       0   
                          CLF3|1200C   0.30   0.30   0.30   0.30       0   
                          CLF3|1400C   0.14   0.14   0.14   0.14       0   
                          CLF3|750C    3.44   3.44   3.44   3.44       0   
                          CLF3|775C    2.86   2.86   2.86   2.86       0   
...                                     ...    ...    ...    ...     ...   
2022-06-01 00:00:00+00:00 CLZ3|920C   13.41  13.41  13.41  13.41       0   
                          CLZ3|930C   13.00  13.00  13.00  13.00       0   
                          CLZ3|940C   12.59  12.59  12.59  12.59       0   
                          CLZ3|950C   12.19  12.15  12.19  12.19       2   
                          CLZ3|990C   10.71  10.71  10.71  10.71       0   

                                      OpenInterest  
Date_Time                 ID                        
2021-09-09 00:00:00+00:00 CLF3|1000C             8  
                          CLF3|1200C           200  
                          CLF3|1400C           200  
                          CLF3|750C              1  
                          CLF3|775C              3  
...                                            ...  
2022-06-01 00:00:00+00:00 CLZ3|920C              7  
                          CLZ3|930C              6  
                          CLZ3|940C              1  
                          CLZ3|950C           2673  
                          CLZ3|990C              1  

[204775 rows x 6 columns]

In [8]:
# Local Options
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
data = mds.get("WTICrudeOil")
data

Open   High    Low  Close  Volume  \
Date_Time                 ID                                             
2019-12-30 00:00:00+00:00 CLZ2|540C  6.23   6.23   6.23   6.23       0   
2019-12-31 00:00:00+00:00 CLZ2|540C  6.13   6.13   6.13   6.13       0   
2020-01-01 00:00:00+00:00 CLZ2|540C   NaN    NaN    NaN    NaN       0   
2020-01-02 00:00:00+00:00 CLZ2|540C  6.35   6.35   6.35   6.35       0   
2020-01-03 00:00:00+00:00 CLZ2|540C  6.21   6.21   6.21   6.21       0   
...                                   ...    ...    ...    ...     ...   
2022-10-09 00:00:00+00:00 CLZ3|940C  0.00   9.19   9.19   9.19       0   
                          CLZ3|945C  0.00   9.04   9.04   9.04       0   
                          CLZ3|950C  0.00   9.26   8.89   8.89     500   
                          CLZ3|950P  0.00  24.39  24.23  24.39       0   
                          CLZ3|990C  0.00   7.76   7.76   7.76       0   

                                     OpenInterest  
Date_Time                 ID                       
2019-12-30 00:00:00+00:00 CLZ2|540C           100  
2019-12-31 00:00:00+00:00 CLZ2|540C           100  
2020-01-01 00:00:00+00:00 CLZ2|540C             0  
2020-01-02 00:00:00+00:00 CLZ2|540C           100  
2020-01-03 00:00:00+00:00 CLZ2|540C           100  
...                                           ...  
2022-10-09 00:00:00+00:00 CLZ3|940C             1  
                          CLZ3|945C            82  
                          CLZ3|950C          5634  
                          CLZ3|950P             5  
                          CLZ3|990C             1  

[515053 rows x 6 columns]

In [14]:
# Remote (cluster)
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
mds.append("WTICrudeOil", data["2022-06-01":], "D", update=True, debug=True)

http://pricestore.192.168.1.203.nip.io/prices/datasource/WTICrudeOil?unit=D
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/WTICrudeOil?unit=D
{'rc': 'success'}


{'rc': 'success'}

In [8]:
data = fetchHistoricalData(None, "../datasources/datasources_BarChart.json")

100%|██████████| 1/1 [00:05<00:00,  5.26s/it]

Adding CLX22 to WTICrudeOil table


In [12]:
if not hasattr(data.index.get_level_values("Date_Time"), "tz") or data.index.get_level_values("Date_Time").tz is None:
    print("here")

here


In [11]:
# Update options config file with all option chains
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))
market = datasources[0]["markets"][0]

# Get Chains
chains = getOptionChains(datasources[0], market["optionRoot"])
market["optionChains"].extend(json.loads(chains.to_json(orient="records")))

# Get Options and Sources from Chains
options, datasources = fetchOptionData(datasources)

#with open(ds_file, 'w', encoding='utf-8') as f:
#    json.dump(datasources, f, ensure_ascii=False, indent=4)
    

In [8]:
# Local Options
mds = MarketDataStore(location="../datastore")
options = fetchHistoricalData(mds, "../datasources/datasources_BarChartOption.json", records=300, newOnly=True)




  0%|          | 0/3117 [00:00<?, ?it/s]

Converting from UTC to UTC
Adding CLG4|1300C to Oil - US Crude table
Request to add data to table: CLG4|1300C
Resampling to D periods
Creating new table for data...
Update complete





 99%|█████████▉| 3093/3117 [00:07<00:00, 427.05it/s]

Converting from UTC to UTC
Adding CLG4|1350C to Oil - US Crude table
Request to add data to table: CLG4|1350C
Resampling to D periods
Creating new table for data...
Update complete
Converting from UTC to UTC
Adding CLG4|1400C to Oil - US Crude table
Request to add data to table: CLG4|1400C
Resampling to D periods
Creating new table for data...
Update complete
Converting from UTC to UTC
Adding CLG4|1410C to Oil - US Crude table
Request to add data to table: CLG4|1410C
Resampling to D periods
Creating new table for data...
Update complete





 99%|█████████▉| 3093/3117 [00:26<00:00, 427.05it/s]

...




100%|█████████▉| 3115/3117 [02:27<00:13,  6.56s/it]

Converting from UTC to UTC
Adding CLK4|1500C to Oil - US Crude table
Request to add data to table: CLK4|1500C
Resampling to D periods
Creating new table for data...
Update complete





100%|█████████▉| 3116/3117 [02:34<00:06,  6.96s/it]

Converting from UTC to UTC
Adding CLK4|1550C to Oil - US Crude table
Request to add data to table: CLK4|1550C
Resampling to D periods
Creating new table for data...
Update complete





100%|██████████| 3117/3117 [02:40<00:00,  6.53s/it]




In [4]:
# Local Options
mds = MarketDataStore(location="../datastore")
options = appendOptionChainPrices(mds, "../datasources/datasources_BarChartOption.json")

100%|██████████| 21/21 [01:27<00:00,  3.72s/it]

Adding CLK24 to WTICrudeOil table


In [7]:
mds.append("WTICrudeOil", options, "D", update=True, debug=True)

Request to add data to table: WTICrudeOil
Table found: WTICrudeOil
Appending data...
Update complete


In [12]:
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))

mkt = datasources[0]["markets"][0]
srcs = []
options = []
for source in mkt["sources"]:
    srcs.append({"name": source["name"], "sample_unit": source["sample_unit"]})
    strike = float(source["name"][5:-1])
    type = source["name"][-1].lower()
    if type == 'c':
        typeString = "Call"
    else:
        typeString = "Put"
    options.append({
        "epic": source["name"],
        "instrumentName": "Crude Oil WTI Oct '22 {} {}".format(strike, typeString),
        "strike": strike,
        "type": type
    })

mkt["sources"] = srcs
mkt["optionChains"][0]["options"] = options

#with open(ds_file, 'w', encoding='utf-8') as f:
#    json.dump(datasources, f, ensure_ascii=False, indent=4)

In [15]:
#fetchHistoricalData(None, "../datasources/datasources_MDS.json", end="2022-09-15")
ds_file = "../datasources/datasources_MDS.json"

options, datasources = fetchOptionData(json.load(open(ds_file)), end="2022-09-15")

  0%|          | 0/1 [00:00<?, ?it/s]

Loading data from CLV22 in ../datastore/data.hdf
Resampling to D periods
Resampling to D periods
Adding CLV2|210P to Option CLV22
Adding CLV2|220P to Option CLV22
Adding CLV2|230P to Option CLV22
Adding CLV2|240P to Option CLV22
Adding CLV2|250P to Option CLV22
Adding CLV2|260P to Option CLV22
Adding CLV2|270P to Option CLV22
Adding CLV2|275P to Option CLV22
Adding CLV2|280P to Option CLV22
Adding CLV2|290P to Option CLV22
Adding CLV2|300P to Option CLV22
Adding CLV2|310P to Option CLV22
Adding CLV2|315P to Option CLV22
Adding CLV2|320P to Option CLV22
Adding CLV2|325P to Option CLV22
Adding CLV2|330P to Option CLV22
Adding CLV2|340P to Option CLV22
Adding CLV2|345P to Option CLV22
Adding CLV2|350P to Option CLV22
Adding CLV2|355P to Option CLV22
Adding CLV2|360P to Option CLV22
Adding CLV2|365P to Option CLV22
Adding CLV2|370P to Option CLV22
Adding CLV2|375P to Option CLV22
Adding CLV2|380P to Option CLV22
Adding CLV2|390P to Option CLV22
Adding CLV2|395P to Option CLV22
Adding CLV2|

Adding CLV2|1050P to Option CLV22
Adding CLV2|1055P to Option CLV22
Adding CLV2|1055C to Option CLV22
Adding CLV2|1060C to Option CLV22
Adding CLV2|1060P to Option CLV22
Adding CLV2|1065C to Option CLV22
Adding CLV2|1065P to Option CLV22
Adding CLV2|1070C to Option CLV22
Adding CLV2|1070P to Option CLV22
Adding CLV2|1075C to Option CLV22
Adding CLV2|1075P to Option CLV22
Adding CLV2|1080C to Option CLV22
Adding CLV2|1080P to Option CLV22
Adding CLV2|1085P to Option CLV22
Adding CLV2|1085C to Option CLV22
Adding CLV2|1090P to Option CLV22
Adding CLV2|1090C to Option CLV22
Adding CLV2|1095C to Option CLV22
Adding CLV2|1095P to Option CLV22
Adding CLV2|1100C to Option CLV22
Adding CLV2|1100P to Option CLV22
Adding CLV2|1105C to Option CLV22
Adding CLV2|1105P to Option CLV22
Adding CLV2|1110C to Option CLV22
Adding CLV2|1110P to Option CLV22
Adding CLV2|1115C to Option CLV22
Adding CLV2|1115P to Option CLV22
Adding CLV2|1120C to Option CLV22
Adding CLV2|1120P to Option CLV22
Adding CLV2|11

100%|██████████| 1/1 [02:26<00:00, 146.49s/it]


In [35]:
options.to_pickle("tmp.pkl")

In [36]:
import plotly.express as px
df = px.data.gapminder()

In [ ]:
# Local Options
mds = MarketDataStore(location="../datastore")
options = fetchHistoricalData(mds, "../datasources/datasources_BarChartOption.json", records=300, newOnly=True)

In [6]:
# Local
mds = MarketDataStore(location="../datastore")
#print(mds.getKeys())
#fetchHistoricalData(mds, "../datasources/datasources.json")
#refreshDatasources(mds, "../datasources/datasources_BarChart.json", start="2022-09-01", end="2022-09-020")

In [15]:
# Remote (cluster)
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
#fetchHistoricalData(mds, "../datasources", "datasources.json")

In [ ]:
# Remote (localhost)
mds = MarketDataStore(remote=True, location="http://localhost:8080")
refreshMarketData(mds, "../datasources", "datasources.json")

In [11]:
df2 = mds.aggregate(["D&J-IND","SANDP-500"], "H", "1979-01-01", "2050-01-01", debug=True)
df2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Open,High,Low,Close
2013-01-02 15:00:00+00:00,13366.480000,13374.820000,13338.680000,13345.450000
2013-01-02 16:00:00+00:00,13345.600000,13353.750000,13321.640000,13321.770000
2013-01-02 17:00:00+00:00,13321.760000,13339.250000,13320.280000,13326.660000
2013-01-02 18:00:00+00:00,13326.640000,13336.060000,13322.510000,13329.190000
2013-01-02 19:00:00+00:00,13329.090000,13339.820000,13318.190000,13331.800000
...,...,...,...,...
2018-03-19 16:00:00+00:00,24632.111675,24650.292289,24554.939070,24583.990050
2018-03-19 17:00:00+00:00,24579.349894,24585.490101,24473.096306,24568.679533
2018-03-19 18:00:00+00:00,24570.309589,24590.980286,24455.305705,24533.498346
2018-03-19 19:00:00+00:00,24535.438411,24638.041875,24492.786971,24623.971400


In [8]:
df1 = mds.aggregate(["D&J-IND","SANDP-500"], "H", "1979-01-01", "2050-01-01", debug=True)
df1

Loading data from D&J-IND in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods
Loading data from SANDP-500 in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods


,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.480000,13374.820000,13338.680000,13345.450000
2013-01-02 16:00:00+00:00,13345.600000,13353.750000,13321.640000,13321.770000
2013-01-02 17:00:00+00:00,13321.760000,13339.250000,13320.280000,13326.660000
2013-01-02 18:00:00+00:00,13326.640000,13336.060000,13322.510000,13329.190000
2013-01-02 19:00:00+00:00,13329.090000,13339.820000,13318.190000,13331.800000
...,...,...,...,...
2018-03-19 16:00:00+00:00,24632.111675,24650.292289,24554.939070,24583.990050
2018-03-19 17:00:00+00:00,24579.349894,24585.490101,24473.096306,24568.679534
2018-03-19 18:00:00+00:00,24570.309589,24590.980286,24455.305705,24533.498346
